In [13]:
from gensim.models.doc2vec import Doc2Vec
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

In [1]:
def clean(text):
    text = re.sub(r'\|\|\|', r' ', text)
    text = text.replace('„','')
    text = text.replace('“','')
    text = text.replace('"','')
    text = text.replace('\'','')
    text = text.replace('-',' ')
    text = text.replace('&', 'and')
    text = text.lower()
    return text

def remove_stopwords(text):
    engl_stops = set(stopwords.words('english'))
    return ' '.join([word for word in text.split() if word not in engl_stops])

def tokenize(text):
    return list(filter(lambda word : len(word) > 3, word_tokenize(text)))

def preprocess(text):
    return tokenize(remove_stopwords(clean(text)))

In [3]:
vec_model = Doc2Vec.load('course_embeddings.model')

In [16]:
def gen_recs(inp, num_recs=10):
    print(preprocess(inp))
    inf = vec_model.infer_vector(preprocess(inp))
    sims = vec_model.dv.most_similar([inf], topn=100)
    titles = set()
    satisfied = 0
    for i, sim in sims:
        course = cmp_classes[i]
        if course[2] not in titles:
            titles.add(course[2])
            satisfied += 1
            print(sim)
            print(course)
            if satisfied == num_recs:
                break
    print(len(sims))

In [19]:
gen_recs("""
Join us for a unique, 12-week, paid, full-time internship for students who are interested in working in technology and full-time opportunities at Google. Relocation assistance is available on a case by case basis. This program will give you an opportunity to work on complex computer science solutions, develop scalable, distributed software systems, and also collaborate on multitudes of smaller projects that have universal appeal, which requires research, awareness, interactivity, and asking questions.

Google is and always will be an engineering company. We hire people with a broad set of technical skills who are ready to address some of technology's greatest challenges and make an impact on millions, if not billions, of users. At Google, engineers not only revolutionize search, they routinely work on massive scalability and storage solutions, large-scale applications and entirely new platforms for developers around the world. From Google Ads to Chrome, Android to YouTube, Social to Local, Google engineers are changing the world one technological achievement after another.
""")

['join', 'unique', 'week', 'paid', 'full', 'time', 'internship', 'students', 'interested', 'working', 'technology', 'full', 'time', 'opportunities', 'google', 'relocation', 'assistance', 'available', 'case', 'case', 'basis', 'program', 'give', 'opportunity', 'work', 'complex', 'computer', 'science', 'solutions', 'develop', 'scalable', 'distributed', 'software', 'systems', 'also', 'collaborate', 'multitudes', 'smaller', 'projects', 'universal', 'appeal', 'requires', 'research', 'awareness', 'interactivity', 'asking', 'questions', 'google', 'always', 'engineering', 'company', 'hire', 'people', 'broad', 'technical', 'skills', 'ready', 'address', 'technologys', 'greatest', 'challenges', 'make', 'impact', 'millions', 'billions', 'users', 'google', 'engineers', 'revolutionize', 'search', 'routinely', 'work', 'massive', 'scalability', 'storage', 'solutions', 'large', 'scale', 'applications', 'entirely', 'platforms', 'developers', 'around', 'world', 'google', 'chrome', 'android', 'youtube', 's

NameError: name 'cmp_classes' is not defined